In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
from IPython.display import display

year = "111"
semester = "1"
dataLink = "https://shcourse.utaipei.edu.tw"

grade = {
    1: "1411",
    2: "1421",
    3: "1431",
    4: "1441",
    5: "1611",  # 碩一
    6: "1621",  # 碩二
    7: "4611",  # 職碩一
    8: "4621"  # 職碩二
}

department = [
    "8900", "8910", "9000", "9002", "9003", "9004", "9100", "9200", "9300",
    "9652", "9668", "9674", "9750", "2600", "7100", "7200", "7202", "7205",
    "7206", "7300", "7400", "7500", "7600", "9671", "9689", "9747", "9748",
    "9749", "9751", "9752", "9753", "XS00", "5700", "5710", "7900", "8100",
    "8200", "8300", "8400", "8500", "8800", "9612", "9634", "9641", "9681",
    "5000", "5100", "5200", "5300", "5500", "5510", "5611", "5800", "5900",
    "6000", "6201", "6400", "6600", "6700", "ZZ40", "9706", "9707", "9725",
    "9737", "9738", "9739", "9741", "ZZ80", "2410", "2421", "2422", "9705",
    "9708", "9726"
]

file_name = year + semester + "0-data.json"
f = open(file_name, 'w')
f.write('{')
f.close()


In [2]:
# 修正ssl錯誤的問題
# 學校於2022/09/05 強制https，導致python無法連線
import ssl
from urllib3 import poolmanager

class TLSAdapter(requests.adapters.HTTPAdapter):

    def init_poolmanager(self, connections, maxsize, block=False):
        """Create and initialize the urllib3 PoolManager."""
        ctx = ssl.create_default_context()
        ctx.set_ciphers('DEFAULT@SECLEVEL=1')
        self.poolmanager = poolmanager.PoolManager(
            num_pools=connections,
            maxsize=maxsize,
            block=block,
            ssl_version=ssl.PROTOCOL_TLS,
            ssl_context=ctx)


session = requests.session()
session.mount('https://', TLSAdapter())
session.keep_alive=False

In [3]:
def parse_course_info(raw_data):
    inputerror = 0
    try:
        raw_data = raw_data.replace('\r', '')
        raw_data = raw_data.replace('\n', '')
        raw_data = raw_data.replace('(單週)', '')
        tmp = raw_data.split("(")
        course_info = np.array(["", "", ""], dtype=object)
        course_info[0] = tmp[0]  # teacher
        course_info[1] = ""
        course_info[2] = ""

        if len(tmp)>3:
            #print("#raw_data :", raw_data)
            #print("tmp :", tmp)
            day = tmp[1].split(")")
            #print("day :", day)
            try:
                time = day[1].split("(")
                time.append(tmp[3].split(")")[1])
            except:
                time = "學校無提供或是機器人發生錯誤:("
                # print("**input error - room **")
            #print("time :", time)
            try:
                room = tmp[4].replace(")", '')
            except:
                room = "學校無提供或是機器人發生錯誤:("
                # print("**input error - room **")
            #print("room :", room)
            course_info[0] += "、" + tmp[2].split(")")[1]
            

        else:
            #print("raw_data :",raw_data)
            #print("tmp :",tmp)
            day = tmp[1].split(")")
            #print("day :", day)
            try:
                time = day[1].split("(")
            except:
                time = "學校無提供或是機器人發生錯誤:("
                # print("**input error - room **")
            #print("time :", time)
            try:
                room = tmp[2].replace(")", '')
            except:
                room = "學校無提供或是機器人發生錯誤:("
                # print("**input error - room **")
            #print("room :",room)
            time = time[0].split("-")

        
        weekday_eng = {"一": "M",
                       "二": "T",
                       "三": "W",
                       "四": "R",
                       "五": "F",
                       "六": "S"}

        for i in range(int(time[0]), int(time[1])+1):
            # print(i)
            if(i == 10):
                course_info[1] += weekday_eng[day[0]] + "a"
            elif(i == 11):
                course_info[1] += weekday_eng[day[0]] + "b"
            elif(i == 12):
                course_info[1] += weekday_eng[day[0]] + "c"
            elif(i == 13):
                course_info[1] += weekday_eng[day[0]] + "d"
            elif(i == 14):
                course_info[1] += weekday_eng[day[0]] + "e"
            else:
                course_info[1] += weekday_eng[day[0]] + str(i)

        course_info[2] = room

    except:
        #print("**input error**")
        inputerror = inputerror + 1

    return course_info



In [4]:
def made_json(prefix, class_table):
    global file_name
    f = open(file_name, 'a')
    for i in range(1, len(class_table)):
        course_json_item = ""
        course_json_item += '"' + prefix + "" + class_table[i][0] + '":{'
        course_json_item += '"' + "id" + '":' + '"' + \
            prefix + "" + class_table[i][0] + '"' + ','
        course_json_item += '"' + "name" + '":' + \
            '"' + class_table[i][1] + '"' + ','
        course_json_item += '"' + "credit" + '":' + \
            '"' + class_table[i][3] + '"' + ','
        # 領域類
        course_json_item += '"' + "field" + '":' + \
            '"' + class_table[i][9] + '"' + ','
        # 校區
        course_json_item += '"' + "area" + '":' + \
            '"' + class_table[i][7] + '"' + ','
        info = parse_course_info(class_table[i][8])
        course_json_item += '"' + "time" + '":' + '"' + info[1] + '"' + ','
        course_json_item += '"' + "room" + '":' + '"' + info[2] + '"' + ','
        course_json_item += '"' + "teacher" + '":' + '"' + info[0] + '"' + ','
        course_json_item += '"' + "english" + '":' + "false" + ','
        course_type = {
            '【選修】': "0",
            '【必修】': "1"
        }
        course_json_item += '"' + "type" + '":' + \
            course_type[class_table[i][5]] + '}'
        #print("parse_course_info :",info)
        # print(course_json_item)
        # print('---------------------------------------------------')
        course_json_item = course_json_item.replace('\n', '')
        f.write("," + course_json_item + "\n")

    f.close()


In [5]:
import time


def parse_html(r, prefix, link):
    class_table = np.empty(([0, 12]), dtype=object)
    soup = BeautifulSoup(r.text, "html.parser")  # 將網頁資料以html.parser
    sel = soup.select("form td")  # 取HTML標中的 <form ></form> 中的<td>標籤存入sel
    tmp = np.empty(12, dtype=object)
    try:
        for j in range(0, len(sel)+1):

            if(j % 12 == 0 and j != 0):
                # print(tmp)
                class_table = np.append(class_table, [tmp], axis=0)
                if(j == len(sel)):
                    break
            tmp[j % 12] = sel[j].text.replace('\xa0', '')
    except:
        soup = BeautifulSoup(r.text, "html.parser")  # 將網頁資料以html.parser
        # 取HTML標中的 <center ></center> 中的<td>標籤存入sel
        sel = soup.select("center")
        if sel[0].text != '查無相符資料。':
            print("**資料解析錯誤**", link)
        # print(r.text)
    # print(class_table)
    made_json(prefix, class_table)
    return


#r = requests.get("http://shcourse.utaipei.edu.tw/utaipei/ag_pro/ag304_03.jsp?arg01=110&arg02=1&arg=91001411&uid=null")
# r = requests.get("http://shcourse.utaipei.edu.tw/utaipei/ag_pro/ag304_03.jsp?arg01=110&arg02=1&arg=XS001400&uid=null") #通識
# print(r.text)

display_working = display('#目前進度:', display_id=True)
display_link = display('#取得資料', display_id=True)

# s = requests.session()
# s.keep_alive = False

link=""
for i in range(0, len(department)):
    for j in range(1, 9):
        link = dataLink + "/utaipei/ag_pro/ag304_03.jsp?arg01="
        link += year + "&arg02=" + semester + "&arg=" + \
            department[i] + grade[j] + "&uid=null"
        # r = ""
        try:
            #r = requests.get(link, verify=False)
            r = session.get(link)
            time.sleep(0.5)
        except:
            print("**連結出現錯誤**", department[i] + grade[j], link)
            continue
        display_link.update("#取得資料成功"+department[i] + grade[j])
        display_working.update("#目前進度:"+str(i/len(department)*100)+"%")
        parse_html(r, year+semester+"0" + department[i] + str(j), link)

r = session.get(dataLink+"/utaipei/ag_pro/ag304_03.jsp?arg01=110&arg02=" +
                 semester+"&arg=XS001400&uid=null")  # 通識
parse_html(r, year+semester+"0" + "XS00" + "1", "通識")

f = open(file_name, 'a')
f.write("}")
f.close()


'#目前進度:98.64864864864865%'

'#取得資料成功97264621'

# 測試程式

In [8]:
def parse_course_info_test(raw_data):
    inputerror = 0
    try:
        raw_data = raw_data.replace('\r', '')
        raw_data = raw_data.replace('\n', '')
        raw_data = raw_data.replace('(單週)', '')
        tmp = raw_data.split("(")
        course_info = np.array(["", "", ""], dtype=object)
        course_info[0] = tmp[0]  # teacher
        course_info[1] = ""
        course_info[2] = ""

        if len(tmp)>3:
            print("#raw_data :", raw_data)
            print("tmp :", tmp)
            day = tmp[1].split(")")
            print("day :", day)
            try:
                time = day[1].split("(")
                time.append(tmp[3].split(")")[1])
            except:
                time = "學校無提供或是機器人發生錯誤:("
                # print("**input error - room **")
            print("time :", time)
            try:
                room = tmp[4].replace(")", '')
            except:
                room = "學校無提供或是機器人發生錯誤:("
                # print("**input error - room **")
            print("room :", room)
            course_info[0] += "、" + tmp[2].split(")")[1]
            

            '''
            raw_data : 劉興政(三)1(C505)邱麗萍(三)2(C505)
            tmp : ['劉興政', '三)1', 'C505)邱麗萍', '三)2', 'C505)']
            day : ['三', '1']
            time : ['1']
            room : C505邱麗萍
            **input error**
            ['劉興政' '' '']

            raw_data : 梁世聰 (二)6-7(G313)
            tmp : ['梁世聰 ', '二)6-7', 'G313)']
            day : ['二', '6-7']
            time : ['6-7']
            room : G313
            ['梁世聰 ' 'T6T7' 'G313']
            '''

        else:
            print("raw_data :",raw_data)
            print("tmp :",tmp)
            day = tmp[1].split(")")
            print("day :", day)
            try:
                time = day[1].split("(")
            except:
                time = "學校無提供或是機器人發生錯誤:("
                # print("**input error - room **")
            print("time :", time)
            try:
                room = tmp[2].replace(")", '')
            except:
                room = "學校無提供或是機器人發生錯誤:("
                # print("**input error - room **")
            print("room :",room)
            time = time[0].split("-")

        
        weekday_eng = {"一": "M",
                       "二": "T",
                       "三": "W",
                       "四": "R",
                       "五": "F",
                       "六": "S"}

        for i in range(int(time[0]), int(time[1])+1):
            # print(i)
            if(i == 10):
                course_info[1] += weekday_eng[day[0]] + "a"
            elif(i == 11):
                course_info[1] += weekday_eng[day[0]] + "b"
            elif(i == 12):
                course_info[1] += weekday_eng[day[0]] + "c"
            elif(i == 13):
                course_info[1] += weekday_eng[day[0]] + "d"
            elif(i == 14):
                course_info[1] += weekday_eng[day[0]] + "e"
            else:
                course_info[1] += weekday_eng[day[0]] + str(i)

        course_info[2] = room

    except:
        print("**input error**")
        inputerror = inputerror + 1

    return course_info


link = "http://shcourse.utaipei.edu.tw/utaipei/ag_pro/ag304_03.jsp?arg01=110&arg02=2&arg=91001411&uid=null"
#link = "http://shcourse.utaipei.edu.tw/utaipei/ag_pro/ag304_03.jsp?arg01=110&arg02=2&arg=97061611&uid=null"

r = ""
r = requests.get(link)
class_table = np.empty(([0, 12]), dtype=object)
soup = BeautifulSoup(r.text, "html.parser")  # 將網頁資料以html.parser
sel = soup.select("form td")  # 取HTML標中的 <form ></form> 中的<td>標籤存入sel
tmp = np.empty(12, dtype=object)
try:
    for j in range(0, len(sel)+1):

        if(j % 12 == 0 and j != 0):
            # print(tmp)
            class_table = np.append(class_table, [tmp], axis=0)
            if(j == len(sel)):
                break
        tmp[j % 12] = sel[j].text.replace('\xa0', '')
except:
    soup = BeautifulSoup(r.text, "html.parser")  # 將網頁資料以html.parser
    sel = soup.select("center")  # 取HTML標中的 <form ></form> 中的<td>標籤存入sel
    if sel[0].text != '查無相符資料。':
        print("**資料解析錯誤**", link)
    else:
        print(sel[0].text)
    # print(r.text)
#print(class_table)
print(parse_course_info_test(class_table[1][8]))







SSLError: HTTPSConnectionPool(host='shcourse.utaipei.edu.tw', port=443): Max retries exceeded with url: /utaipei/ag_pro/ag304_03.jsp?arg01=110&arg02=2&arg=91001411&uid=null (Caused by SSLError(SSLError(1, '[SSL: WRONG_SIGNATURE_TYPE] wrong signature type (_ssl.c:997)')))

In [6]:
print(parse_course_info_test("劉興政(三)1(C505)邱麗萍(三)2(C505)"))


#raw_data : 劉興政(三)1(C505)邱麗萍(三)2(C505)
tmp : ['劉興政', '三)1', 'C505)邱麗萍', '三)2', 'C505)']
day : ['三', '1']
time : ['1', '2']
room : C505
['劉興政、邱麗萍' 'W1W2' 'C505']


# 雜七雜八的東東

```
f = open(file_name, 'a')
for i in range(1,len(class_table)):
    course_json_item = ""
    course_json_item += '"' + "1101091001 " +class_table[i][0] + '":{'
    course_json_item += '"' + "id" + '":' + '"' + "1101091001 " + class_table[i][0] + '"' + ','
    course_json_item += '"' + "name" + '":' + '"' + class_table[i][1] + '"' + ','
    course_json_item += '"' + "credit" + '":' + '"' + class_table[i][3] + '"' + ','
    info = parse_course_info(class_table[i][8])
    course_json_item += '"' + "time" + '":' + '"' + info[1] + '"' + ','
    course_json_item += '"' + "room" + '":' + '"' + info[2] + '"' + ','
    course_json_item += '"' + "teacher" + '":' + '"' + info[0] + '"' + ','
    course_json_item += '"' + "english" + '":'  + "false" + ','
    course_type = {
        '【選修】':"0",
        '【必修】':"1"
    }
    course_json_item += '"' + "type" + '":'  + course_type[class_table[i][5]] + '}'
    print("parse_course_info :",info)
    print(course_json_item)
    print('---------------------------------------------------')
    if(i==len(class_table)-1):
        f.write(course_json_item)
        continue
    f.write(course_json_item + ",\n")
f.write("}")
f.close()




soup = BeautifulSoup(r.text,"html.parser") #將網頁資料以html.parser
sel = soup.select("form td") #取HTML標中的 <form ></form> 中的<td>標籤存入sel

#for s in sel:
tmp = np.empty(12,dtype=object)
for j in range(0,len(sel)+1):
    if(j%12==0 and j!=0):
        #print(tmp)
        class_table = np.append(class_table, [tmp], axis=0)
        if(j==len(sel)):
            break
    tmp[j%12]=sel[j].text.replace('\xa0','')
print(class_table)



<option value="24">師資培育中心</option>
<option value="51">進修推廣處</option>
<option value="UZ">市政管理學院</option>
<option value="WZ">教育學院</option>
<option value="XS">校共同科目</option>
<option value="XZ">人文藝術學院</option>
<option value="YZ">理學院</option>
<option value="ZZ">體育學院</option></select>


<select id="unt_id" name="unt_id" size="1" class="text11">
<option value="8900">地球環境暨生物資源學系</option>
<option value="8910">地球環境暨生物資源學系環境教育碩士在職專班</option>
<option value="9000">應用物理暨化學系</option>
<option value="9002">應用物理暨化學系電子物理組</option>
<option value="9003">應用物理暨化學系應用化學組</option>
<option value="9004">應用物理暨化學系應用科學碩士班</option>
<option value="9100">資訊科學系</option>
<option value="9200">數學系</option>
<option value="9300">體育學系</option>
<option value="9652">數學系數學教育碩士在職專班</option>
<option value="9668">數學系數據科學與數學教育碩士班</option>
<option value="9674">體育學系碩士在職專班</option>
<option value="9750">應用物理暨化學系應用科學碩士在職專班</option></select>

<select id="unt_id" name="unt_id" size="1" class="text11">
<option value="9747">城市發展學系</option>
<option value="9748">都會產業經營與行銷學系</option>
<option value="9749">衛生福利學系</option>
<option value="9751">城市發展學系碩士班</option>
<option value="9752">都會產業經營與行銷學系碩士班</option>
<option value="9753">衛生福利學系碩士班</option></select>

<select id="unt_id" name="unt_id" size="1" class="text11">
<option value="7100">教育學系</option>
<option value="7200">特殊教育學系</option>
<option value="7202">特殊教育學系碩士在職專班</option>
<option value="7205">特殊教育學系碩士班語言治療組</option>
<option value="7206">特殊教育學系碩士班特殊教育組</option>
<option value="7300">幼兒教育學系</option>
<option value="7400">心理與諮商學系</option>
<option value="7500">學習與媒材設計學系</option>
<option value="7600">教育行政與評鑑研究所</option>
<option value="9671">教育行政與評鑑研究所碩士在職專班</option>
<option value="9689">學習與媒材設計學系課程與教學碩士學位在職專班</option></select>

<select id="unt_id" name="unt_id" size="1" class="text11">
<option value="5700">舞蹈學系</option>
<option value="5710">舞蹈學系碩士班</option>
<option value="7900">中國語文學系</option>
<option value="8100">歷史與地理學系</option>
<option value="8200">音樂學系</option>
<option value="8300">視覺藝術學系</option>
<option value="8400">英語教學系</option>
<option value="8500">社會暨公共事務學系</option>
<option value="8800">華語文教學碩士學位學程</option>
<option value="9612">中國語文學系碩士在職專班</option>
<option value="9634">音樂學系碩士在職專班</option>
<option value="9641">視覺藝術學系碩士在職專班</option>
<option value="9681">社會暨公共事務學系碩士在職專班</option></select>

<select id="unt_id" name="unt_id" size="1" class="text11">
<option value="5000">球類運動學系</option>
<option value="5100">陸上運動學系</option>
<option value="5200">水上運動學系</option>
<option value="5300">技擊運動學系</option>
<option value="5500">休閒運動管理學系</option>
<option value="5510">休閒運動管理學系碩士班</option>
<option value="5611">運動健康科學系碩士班</option>
<option value="5800">運動科學研究所</option>
<option value="5900">運動教育研究所</option>
<option value="6000">運動器材科技研究所</option>
<option value="6201">身心障礙者轉銜及休閒教育碩士學位學程</option>
<option value="6400">競技運動訓練研究所</option>
<option value="6600">運動藝術學系</option>
<option value="6700">運動健康科學系</option>
<option value="ZZ40">校際選課</option></select>
```